# Notebook 1
This notebook is used for EDA and to build an initial model using Pytorch framework

Benny Lee - 13371063

In [1]:
%load_ext autoreload
%autoreload 2

# Load packages and data

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

In [3]:
df = pd.read_csv('../data/raw/beer_reviews.csv')

# Exploration

In [4]:
df.describe()

brewery_id   review_time  review_overall  review_aroma  \
count  1.586614e+06  1.586614e+06    1.586614e+06  1.586614e+06   
mean   3.130099e+03  1.224089e+09    3.815581e+00  3.735636e+00   
std    5.578104e+03  7.654427e+07    7.206219e-01  6.976167e-01   
min    1.000000e+00  8.406720e+08    0.000000e+00  1.000000e+00   
25%    1.430000e+02  1.173224e+09    3.500000e+00  3.500000e+00   
50%    4.290000e+02  1.239203e+09    4.000000e+00  4.000000e+00   
75%    2.372000e+03  1.288568e+09    4.500000e+00  4.000000e+00   
max    2.800300e+04  1.326285e+09    5.000000e+00  5.000000e+00   

       review_appearance  review_palate  review_taste      beer_abv  \
count       1.586614e+06   1.586614e+06  1.586614e+06  1.518829e+06   
mean        3.841642e+00   3.743701e+00  3.792860e+00  7.042387e+00   
std         6.160928e-01   6.822184e-01  7.319696e-01  2.322526e+00   
min         0.000000e+00   1.000000e+00  1.000000e+00  1.000000e-02   
25%         3.500000e+00   3.500000e+00  3.500000e+00  5.200000e+00   
50%         4.000000e+00   4.000000e+00  4.000000e+00  6.500000e+00   
75%         4.000000e+00   4.000000e+00  4.500000e+00  8.500000e+00   
max         5.000000e+00   5.000000e+00  5.000000e+00  5.770000e+01   

        beer_beerid  
count  1.586614e+06  
mean   2.171279e+04  
std    2.181834e+04  
min    3.000000e+00  
25%    1.717000e+03  
50%    1.390600e+04  
75%    3.944100e+04  
max    7.731700e+04

In [5]:
df.head()

brewery_id             brewery_name  review_time  review_overall  \
0       10325          Vecchio Birraio   1234817823             1.5   
1       10325          Vecchio Birraio   1235915097             3.0   
2       10325          Vecchio Birraio   1235916604             3.0   
3       10325          Vecchio Birraio   1234725145             3.0   
4        1075  Caldera Brewing Company   1293735206             4.0   

   review_aroma  review_appearance review_profilename  \
0           2.0                2.5            stcules   
1           2.5                3.0            stcules   
2           2.5                3.0            stcules   
3           3.0                3.5            stcules   
4           4.5                4.0     johnmichaelsen   

                       beer_style  review_palate  review_taste  \
0                      Hefeweizen            1.5           1.5   
1              English Strong Ale            3.0           3.0   
2          Foreign / Export Stout            3.0           3.0   
3                 German Pilsener            2.5           3.0   
4  American Double / Imperial IPA            4.0           4.5   

                beer_name  beer_abv  beer_beerid  
0            Sausa Weizen       5.0        47986  
1                Red Moon       6.2        48213  
2  Black Horse Black Beer       6.5        48215  
3              Sausa Pils       5.0        47969  
4           Cauldron DIPA       7.7        64883

In [6]:
df.shape

(1586614, 13)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


**Generate data profile report**

In [34]:
from ydata_profiling import ProfileReport


In [9]:
profile = ProfileReport(df, title="Profiling Report")
#profile.to_widgets()
#profile.to_notebook_iframe()

In [10]:
profile.to_file("../reports/beer_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

**Target variable (beer_style)**

In [11]:
import matplotlib.pyplot as plt
import seaborn

total = df['beer_style'].value_counts().values.sum()

def fmt(x):
    return '{:.1f}%\n{:.0f}'.format(x, total*x/100)

plt.pie(df['beer_style'].value_counts().values, labels=df['beer_style'].value_counts().index, autopct=fmt)

plt.show()

In [12]:
nunique_vals = list()

for column in df:
    nunique_vals.append(df[column].nunique())
    
pd.DataFrame({'columns': df.columns,
              'Number of unique values': nunique_vals})


columns  Number of unique values
0           brewery_id                     5840
1         brewery_name                     5742
2          review_time                  1577960
3       review_overall                       10
4         review_aroma                        9
5    review_appearance                       10
6   review_profilename                    33387
7           beer_style                      104
8        review_palate                        9
9         review_taste                        9
10           beer_name                    56857
11            beer_abv                      530
12         beer_beerid                    66055

In [13]:
count_beer_style = df['beer_style'].value_counts()
df_count_beer_style = pd.DataFrame(count_beer_style)

df_count_beer_style['percentage'] = round(df_count_beer_style['beer_style']/df_count_beer_style.beer_style.sum()*100,1)
df_count_beer_style.head(15)

beer_style  percentage
American IPA                          117586         7.4
American Double / Imperial IPA         85977         5.4
American Pale Ale (APA)                63469         4.0
Russian Imperial Stout                 54129         3.4
American Double / Imperial Stout       50705         3.2
American Porter                        50477         3.2
American Amber / Red Ale               45751         2.9
Belgian Strong Dark Ale                37743         2.4
Fruit / Vegetable Beer                 33861         2.1
American Strong Ale                    31945         2.0
Belgian Strong Pale Ale                31490         2.0
Saison / Farmhouse Ale                 31480         2.0
American Adjunct Lager                 30749         1.9
Tripel                                 30328         1.9
Witbier                                30140         1.9

**Brewery Name**

In [14]:
 print(df.brewery_name.nunique())

5742


In [15]:
count_brewery_name = df['brewery_name'].value_counts()
df_count_brewery_name = pd.DataFrame(count_brewery_name)

df_count_brewery_name['percentage'] = round(df_count_brewery_name['brewery_name']/df_count_brewery_name.brewery_name.sum()*100,1)
df_count_brewery_name.head(10)

brewery_name  percentage
Boston Beer Company (Samuel Adams)         39444         2.5
Dogfish Head Brewery                       33839         2.1
Stone Brewing Co.                          33066         2.1
Sierra Nevada Brewing Co.                  28751         1.8
Bell's Brewery, Inc.                       25191         1.6
Rogue Ales                                 24083         1.5
Founders Brewing Company                   20004         1.3
Victory Brewing Company                    19479         1.2
Lagunitas Brewing Company                  16837         1.1
Avery Brewing Company                      16107         1.0

# Prepare Data

In [16]:
# Only copy these features that are relevant to the api
df_cleaned = df[['brewery_name','review_aroma','review_appearance','review_palate','review_taste','beer_abv','beer_style']].copy()
target_col="beer_style"

df_cleaned.columns

Index(['brewery_name', 'review_aroma', 'review_appearance', 'review_palate',
       'review_taste', 'beer_abv', 'beer_style'],
      dtype='object')

**Drop rows contain num value**

In [17]:
df_cleaned.dropna(inplace=True)
df_cleaned.isnull().sum()

brewery_name         0
review_aroma         0
review_appearance    0
review_palate        0
review_taste         0
beer_abv             0
beer_style           0
dtype: int64

In [18]:
# Use the first 5000 row for development.
df_cleaned=df_cleaned.iloc[:5000].copy()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 5218
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brewery_name       5000 non-null   object 
 1   review_aroma       5000 non-null   float64
 2   review_appearance  5000 non-null   float64
 3   review_palate      5000 non-null   float64
 4   review_taste       5000 non-null   float64
 5   beer_abv           5000 non-null   float64
 6   beer_style         5000 non-null   object 
dtypes: float64(5), object(2)
memory usage: 312.5+ KB


**Convert categorical variables to numeric**

In [19]:
brewery_name_list = list(df_cleaned.brewery_name.unique())
beer_style_list = list(df_cleaned.beer_style.unique())

cat_dict = {
    'brewery_name': [brewery_name_list],
    'beer_style': [beer_style_list]
}

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
for col, valueList in cat_dict.items():
    col_encoder = OrdinalEncoder(categories=valueList)
    df_cleaned[col] = col_encoder.fit_transform(df_cleaned[[col]])

df_cleaned['beer_style'] = df_cleaned['beer_style'].astype(int)
X = df_cleaned
X.describe()

brewery_name  review_aroma  review_appearance  review_palate  \
count   5000.000000   5000.000000        5000.000000    5000.000000   
mean       7.509000      3.674800           3.724300       3.695100   
std        9.955267      0.789094           0.693207       0.761447   
min        0.000000      1.000000           1.000000       1.000000   
25%        2.000000      3.500000           3.500000       3.500000   
50%        5.000000      4.000000           4.000000       4.000000   
75%        5.000000      4.000000           4.000000       4.000000   
max       56.000000      5.000000           5.000000       5.000000   

       review_taste     beer_abv   beer_style  
count   5000.000000  5000.000000  5000.000000  
mean       3.710500     7.044088    20.285800  
std        0.817602     2.149624    17.978419  
min        1.000000     0.100000     0.000000  
25%        3.500000     5.500000    10.000000  
50%        4.000000     6.800000    14.000000  
75%        4.500000     9.000000    22.000000  
max        5.000000    12.000000    73.000000

**Set target variable to integer**

In [20]:
df_cleaned['beer_style'] = df_cleaned['beer_style'].astype(int)

**Scale numeric features**

In [21]:
num_cols = ['brewery_name', 'review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'beer_abv']
sc = StandardScaler()
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])
X.describe()

brewery_name  review_aroma  review_appearance  review_palate  \
count  5.000000e+03  5.000000e+03       5.000000e+03   5.000000e+03   
mean  -9.094947e-17  1.818989e-16       6.821210e-17  -9.094947e-17   
std    1.000100e+00  1.000100e+00       1.000100e+00   1.000100e+00   
min   -7.543495e-01 -3.390049e+00      -3.930389e+00  -3.539798e+00   
25%   -5.534308e-01 -2.215420e-01      -3.236010e-01  -2.562482e-01   
50%   -2.520526e-01  4.121594e-01       3.977566e-01   4.004617e-01   
75%   -2.520526e-01  4.121594e-01       3.977566e-01   4.004617e-01   
max    4.871376e+00  1.679562e+00       1.840472e+00   1.713882e+00   

       review_taste      beer_abv   beer_style  
count  5.000000e+03  5.000000e+03  5000.000000  
mean  -1.591616e-16 -1.364242e-16    20.285800  
std    1.000100e+00  1.000100e+00    17.978419  
min   -3.315513e+00 -3.230697e+00     0.000000  
25%   -2.574859e-01 -7.183780e-01    10.000000  
50%    3.541196e-01 -1.135605e-01    14.000000  
75%    9.657250e-01  9.099767e-01    22.000000  
max    1.577330e+00  2.305709e+00    73.000000

**Split data into training, validation and test sets**

In [22]:
from src.data.sets import split_sets_random, save_sets
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(X, target_col=target_col, test_ratio=0.2, to_numpy=True)
save_sets(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, path='../data/processed/')

In [23]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

# Define Model architecture

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [25]:
from src.models.pytorch import PytorchMultiClass
model = PytorchMultiClass(X_train.shape[1])

In [26]:
from src.models.pytorch import get_device

device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)

In [27]:
# Print the architecture of model
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)


# Train model

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
N_EPOCHS = 10
BATCH_SIZE = 32

In [29]:
from src.models.pytorch import test_classification, train_classification

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(val)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')


Epoch: 0
	(train)	|	Loss: 0.1333	|	Acc: 41.1%
	(val)	|	Loss: 0.1367	|	Acc: 38.8%
Epoch: 1
	(train)	|	Loss: 0.1323	|	Acc: 43.9%
	(val)	|	Loss: 0.1357	|	Acc: 42.0%
Epoch: 2
	(train)	|	Loss: 0.1321	|	Acc: 44.4%
	(val)	|	Loss: 0.1358	|	Acc: 41.9%
Epoch: 3
	(train)	|	Loss: 0.1324	|	Acc: 43.4%
	(val)	|	Loss: 0.1355	|	Acc: 42.8%
Epoch: 4
	(train)	|	Loss: 0.1320	|	Acc: 44.7%
	(val)	|	Loss: 0.1357	|	Acc: 42.1%
Epoch: 5
	(train)	|	Loss: 0.1322	|	Acc: 44.3%
	(val)	|	Loss: 0.1357	|	Acc: 42.0%
Epoch: 6
	(train)	|	Loss: 0.1320	|	Acc: 44.7%
	(val)	|	Loss: 0.1357	|	Acc: 42.2%
Epoch: 7
	(train)	|	Loss: 0.1321	|	Acc: 44.4%
	(val)	|	Loss: 0.1359	|	Acc: 41.4%
Epoch: 8
	(train)	|	Loss: 0.1322	|	Acc: 44.2%
	(val)	|	Loss: 0.1358	|	Acc: 41.9%
Epoch: 9
	(train)	|	Loss: 0.1323	|	Acc: 43.8%
	(val)	|	Loss: 0.1361	|	Acc: 40.9%


In [30]:
# Save model
torch.save(model, "../models/pytorch.pt")

# Test Model

In [31]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')

	Loss: 0.1354	|	Accuracy: 0.4


In [32]:
from joblib import load
best_model = torch.load("../models/pytorch.pt")

In [33]:
print(best_model)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)
